In [ ]:
# General ASE and Python Modules Modules
import numpy as np
from ase.io import read, write
from ase.visualize import view
from ase import Atoms
from ase.build import molecule
from vasp import Vasp    # John Kitchins Vasp (will change from this when SGE and ASE are integratted)
import os
import glob
import matplotlib.pyplot as plt

import shutil
# GA runner created by Craig
from GA_runner import *


# Load FW and Adsorbate and adjust atom objects as needed

In [ ]:
STRUCT = read('AEI-Si-48T.cif')
ADSORB1 = read('DMDMP-T.cif')
ADSORB2 = read('Ne.cif')
ADSORB = [ADSORB1,ADSORB2]
STRUCT.translate([0,-6,4])    # center the cage near the center of the unit cell
STRUCT.wrap()
NUMS = ADSORB1 + ADSORB2

# Set Initial GA variables

In [ ]:
init_pop_size = 50   # initial structures generated (this number will need to be high, it will be about 1/5 the size once we eliminate bad structures)
n_ads = 2             # number of adsorbates in the ads atom object
Ads_cell = [3,12,3]   # Cell to sample adsorbate in
Ads_cent =[6,7.6,8.5] # center adsorbate at some point in the cage
mult = 1.4            # parameter to multiply natural cuttoffs

# Generate Parents for DB

In [ ]:
GA = GenerateZeoliteConfig(STRUCT,ADSORB,init_pop_size,n_ads)   # Initialize GA
GA.set_ads_cell(Ads_cell)                                       # Apply a unit cell to the adsorbate
GA.construct_init_parent(Ads_cell,Ads_cent)                     # Construct the inital set of parents
GA.remove_overlap(mult)                                         # Eliminate overlapping candidates

# Create initial DB

In [ ]:
# create the database to store information in
db_file = 'gadb_parents.db'

if os.path.isfile(db_file)==False:
    d = PrepareDB(db_file_name=db_file,
                  simulation_cell=STRUCT,
                  stoichiometry=NUMS.get_atomic_numbers())   
    
    for a in GA.Parent_Set:
        d.add_unrelaxed_candidate(a)    # The first element in the DB is the Framework without the Adsorbate
                                        # this is needed for the GA
else:
    print(' ')


# GA and DFT variables

In [ ]:
db = connect(db_file)                    # Connects to the DB
Par_pop_size = db.count()-1              # Number of canidates in the DB
n_to_test = 3                           # Number of children to generate
db_index = range(1,Par_pop_size+1)       # Get the indicies (in python notation (i.e. id=1 has an index=0)
directory = 'Candidates/Candidate-{:02d}/'
def Calc(db_ind,candidate):                               
    return Vasp(directory.format(int(db_ind)),
                xc = 'pbe', 
                encut = 400,                                
                ediff = 1e-5,
                ediffg = -0.03, 
                ibrion = -1,
                isif = 2, 
                nsw = 600,
                ispin = 2,
#                magmom = np.repeat(1,len(candidate)).tolist(),
                nelm = 60,
                sigma = 0.01,
                ismear = 0,
                lreal='A',
                ncore = 24,                                   # set ncore depending on number of cores
                ivdw = 12,
                algo = 'VeryFast',
                atoms = candidate)

# Write DFT input for each Candidate

In [ ]:
# Write input using a cheap optimization method
for i in db_index:                           # write an input file for each job
    parent = read('{0}@{1}'.format(db_file,i))
    calc = Calc(i+1,parent)
    calc.write_input()
    
# Submit Jobs, wait for them to complete before proceeding

# Update DB with Optimized Structures

In [ ]:
### updating may take a few minutes and only needs to be done once
Prep = DB_Prep(db_file,directory,STRUCT,ADSORB,n_ads)
Prep.update_db()
Prep.prep_db()
os.mkdir('DB-Backup')
os.system('cp {0} DB-Backup/'.format(db_file))
db_file2 = 'gadb_gen1.db'
os.system('mv {0} {1}'.format(db_file,db_file2))

# Begin Generation of Children

In [ ]:
# DO NOT RUN THIS BLOCK MULTIPLE TIMES
GA_Gen1 = GenerateZeoliteGen1(Par_pop_size,n_to_test,db_file2)
ddiff = 0.015                        # distance criteria for comparator
dmax = 0.7 
dE = 0.001                           # Energy comparator
GA_Gen1.ga_init(ddiff,dmax,dE)

### If it hangs you may need to delete the DB and start over

# Write input for Generation 1

In [ ]:
# Get the children and optimize those
child_index = range((Par_pop_size+1),(Par_pop_size+1+n_to_test))
for i in child_index:                           # write and input file for each job
    child = read('{0}@{1}'.format(db_file2,i))
    calc = Calc(i+1,child)
    calc.write_input()

# Update DB with Gen1

In [ ]:
### updating may take a few minutes and only needs to be done once
db_file2 = 'gadb_gen1.db'
Prep = DB_Prep(db_file2,directory,STRUCT,ADSORB,n_ads)
Prep.update_db()
Prep.prep_db()
os.system('cp {0} DB-Backup/'.format(db_file2))
db_file3 = 'gadb_gen2.db'
os.system('mv {0} {1}'.format(db_file2,db_file3))

# Begin Generation of Gen2

In [ ]:
# DO NOT RUN THIS BLOCK MULTIPLE TIMES
db_file3 = 'gadb_gen2.db'
new_pop = Par_pop_size + n_to_test
GA_Gen2 = GenerateZeoliteGen2(Par_pop_size,n_to_test,db_file3)
ddiff = 0.015                        # distance criteria for comparator
dmax = 0.7 
dE = 0.001
GA_Gen2.ga_init(ddiff,dmax,dE)

# Write input for Generation 2

In [ ]:
# Get the children and optimize those
new_pop = Par_pop_size + n_to_test
child_index = range((new_pop+1),(new_pop+1+n_to_test))
for i in child_index:                           # write and input file for each job
    child = read('{0}@{1}'.format(db_file3,i))
    calc = Calc(i+1,child)
    calc.write_input()

# Update the DB for the final Time

In [ ]:
### updating may take a few minutes and only needs to be done once
db_file3 = 'gadb_gen2.db'
Prep = DB_Prep(db_file3,directory,STRUCT,ADSORB,n_ads)
Prep.update_db()
Prep.prep_db()
os.system('cp {0} DB-Backup/'.format(db_file3))
db_file4 = 'GA-Final-DB.db'
os.system('mv {0} {1}'.format(db_file3,db_file4))

# Data Analysis

In [ ]:
db_file4 = 'GA-Final-DB.db'    # eneter number of candidates for each generation
gen0 = 28
gen1 = 10
gen2 = 10
DB_EN = DB_viewer(db_file4)
DB_EN.db_get_energies(gen0,gen1,gen2,zoom=True)   # plot out the energy spectrum of each generation